In [89]:
from manager import load_obj, save_obj
from Homework import Homework
from pprint import pprint
from textblob import TextBlob
import csv

from nltk.corpus import stopwords
import re
import nltk
from nltk.tokenize import TweetTokenizer
import string
import numpy as np

from tqdm.notebook import tqdm

from bs4 import BeautifulSoup
from sklearn import metrics
import os

# Chosing bibliography for goldstandard


In [158]:
def clean(txt):
    txt = re.sub(r"[^a-z0-9äöü]", "", txt.lower())
    return txt

In [159]:
def clean_location(txt):
    txt = re.sub(r"[^a-zäöü]", "", txt.lower())
    return txt

In [160]:
def clean_edit(txt):
    txt = re.sub(r"(\s*in\:|hsg\..{1}v|hrsg\.|hsg\.|hsgg\.|hg.|\s*v\.)", "", txt.lower())
    txt = re.sub(r"[^a-z0-9äöü]", "", txt.lower())
    return txt

In [157]:
labels = ["author", "title", "publisher", "location", "editor", "source", "date", "url"]

In [165]:
#credit for the confusion matrix: https://www.educative.io/edpresso/how-to-create-a-confusion-matrix-without-scikit-learn
def confusion_matrix(pred,original): #pass predicted and original labels to this function
    matrix=np.zeros((2,2)) # form an empty matric of 2x2
    
    for val_p,val_o in zip(pred,original): #the confusion matrix is for 2 classes: 1,0
        
        if (val_o != "") and (val_o == val_p):
            matrix[0,0]+=1 #True Positives
        elif (val_o == "") and (val_o != val_p):
            matrix[0,1]+=1 #False Positives
        elif (val_o != "") and (val_o != val_p):
            matrix[1,0]+=1 #False Negatives
        elif (val_o == "") and (val_p ==""):
            matrix[1,1]+=1 #True Negatives
    precision=matrix[0,0]/(matrix[0,0]+matrix[0,1])
    print("Precision:",precision)
    recall=matrix[0,0]/(matrix[0,0]+matrix[1,0])
    print("Recall:",recall)
    f1=2 * ((precision*recall)/(precision+recall))
    print("F1 score:",f1)
    print(matrix)

    #the above code adds up the frequencies of the tps,tns,fps,fns and a matrix is formed
    return matrix

## 1.Anystyle


In [161]:
for val in ground_truth:
    for ref in val:
        if ref.find("journal"):
            print(clean_edit(ref.find("journal").text))

In [162]:


ground_truth =[]
path = os.path.join("C:\\", "Users", "Tim", "Desktop", "bachelorarbeit", "goldstandards","ground_truth")
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r', encoding="utf8") as f: # open in readonly mode
        data = f.read()
        data = BeautifulSoup(data, 'xml') 
        data = data.find_all("sequence")
        ground_truth.append(data)
        
anystyle =[]
path = os.path.join("C:\\", "Users", "Tim", "Desktop", "bachelorarbeit", "goldstandards","anystyle")
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r', encoding="utf8") as f: # open in readonly mode
        data = f.read()
        data = BeautifulSoup(data, 'xml') 
        data = data.find_all("sequence")
        anystyle.append(data)

In [163]:
for i in range(13):
    if len(anystyle[i]) != len(ground_truth[i]):
        print("error" + str(i))
        print(anystyle[i])
        print("\n\n")
        print(ground_truth[i])

In [166]:
#labels
for label in labels:
    lab_any =[]
    lab_ground =[]
    for val_any, val_ground in zip(anystyle, ground_truth):
        for label_any, label_ground in zip(val_any,val_ground):
            tag_a = label_any.find_all(label)
            tag_g = label_ground.find_all(label)
            if tag_a:
                if label == "editor" or label == "source":
                    lab_any.append(clean_edit("".join([word.text for word in tag_a])))
                elif label == "location" or label == "publisher" or label == "author":
                    lab_any.append(clean_location("".join([word.text for word in tag_a])))
                else:
                    lab_any.append(clean("".join([word.text for word in tag_a])))
            else:
                lab_any.append("")
            if tag_g:
                if label == "editor" or label == "source":
                    lab_ground.append(clean_edit("".join([word.text for word in tag_g])))
                elif label == "location" or label == "publisher" or label == "author":
                    lab_ground.append(clean_location("".join([word.text for word in tag_g])))
                else:
                    lab_ground.append(clean("".join([word.text for word in tag_g])))
            else:
                lab_ground.append("")
                
    """
    for i,j in zip(lab_any, lab_ground):
        print("any " + i)
        print("gro " + j)
    """
    print(label+ "\n")
    confusion_matrix(lab_any,lab_ground)
    print("\n")

author

Precision: 0.8947368421052632
Recall: 0.7251184834123223
F1 score: 0.8010471204188482
[[153.  18.]
 [ 58.  29.]]


title

Precision: 0.9
Recall: 0.5294117647058824
F1 score: 0.6666666666666667
[[117.  13.]
 [104.  24.]]


publisher

Precision: 0.7888888888888889
Recall: 0.8068181818181818
F1 score: 0.797752808988764
[[ 71.  19.]
 [ 17. 151.]]


location

Precision: 0.9487179487179487
Recall: 0.6568047337278107
F1 score: 0.7762237762237761
[[111.   6.]
 [ 58.  83.]]


editor

Precision: 0.9722222222222222
Recall: 0.603448275862069
F1 score: 0.7446808510638298
[[ 35.   1.]
 [ 23. 199.]]


source

Precision: 0.7733333333333333
Recall: 0.5420560747663551
F1 score: 0.6373626373626373
[[ 58.  17.]
 [ 49. 134.]]


date

Precision: 0.9946808510638298
Recall: 0.8348214285714286
F1 score: 0.9077669902912622
[[187.   1.]
 [ 37.  33.]]


url

Precision: 1.0
Recall: 0.6
F1 score: 0.7499999999999999
[[  3.   0.]
 [  2. 253.]]




## 2. Grobid


In [167]:

ground_truth =[]
path = os.path.join("C:\\", "Users", "Tim", "Desktop", "bachelorarbeit", "goldstandards","ground_truth")
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r', encoding="utf8") as f: # open in readonly mode
        data = f.read()
        data = BeautifulSoup(data, 'xml') 
        data = data.find_all("sequence")
        ground_truth.append(data)
        
grobid =[]
path = os.path.join("C:\\", "Users", "Tim", "Desktop", "bachelorarbeit", "goldstandards","grobid_xml")
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r', encoding="utf8") as f: # open in readonly mode
        data = f.read()
        data = BeautifulSoup(data, 'xml') 
        data = data.find_all("sequence")
        grobid.append(data)

In [168]:
for i in range(13):
    if len(grobid[i]) != len(ground_truth[i]):
        print("error" + str(i))

In [169]:
#labels
for label in labels:
    lab_grobid =[]
    lab_ground =[]
    for val_grobid, val_ground in zip(grobid, ground_truth):
        for label_grobid, label_ground in zip(val_grobid,val_ground):
            tag_grob = label_grobid.find_all(label)
            tag_g = label_ground.find_all(label)
            
            for i in range(len(tag_grob)):
                if tag_grob[i].text == "":
                    tag_grob.remove(tag_grob[i]) 
            
            
            
            if tag_grob != "":
                if label == "editor" or label == "source":
                    lab_grobid.append(clean_edit("".join([word.text for word in tag_grob])))
                elif label == "location" or label == "publisher" or label == "author":
                    lab_grobid.append(clean_location("".join([word.text for word in tag_grob])))
                else:
                    lab_grobid.append(clean("".join([word.text for word in tag_grob])))
            else:
                lab_grobid.append("")
            if tag_g:
                if label == "editor" or label == "source":
                    lab_ground.append(clean_edit("".join([word.text for word in tag_g])))
                elif label == "location" or label == "publisher" or label == "author":
                    lab_ground.append(clean_location("".join([word.text for word in tag_g])))
                else:
                    lab_ground.append(clean("".join([word.text for word in tag_g])))
            else:
                lab_ground.append("")
    
    """
    for i,j in zip(lab_grobid, lab_ground):
        print("any " + i)
        print("gro " + j)
    """

    print(label+ "\n")
    confusion_matrix(lab_grobid,lab_ground)
    print("\n")

author

Precision: 0.844559585492228
Recall: 0.7725118483412322
F1 score: 0.8069306930693069
[[163.  30.]
 [ 48.  17.]]


title

Precision: 0.8796296296296297
Recall: 0.4298642533936652
F1 score: 0.5775075987841946
[[ 95.  13.]
 [126.  24.]]


publisher

Precision: 0.8723404255319149
Recall: 0.4659090909090909
F1 score: 0.6074074074074074
[[ 41.   6.]
 [ 47. 164.]]


location

Precision: 1.0
Recall: 0.2781065088757396
F1 score: 0.4351851851851852
[[ 47.   0.]
 [122.  89.]]


editor

Precision: 0.8484848484848485
Recall: 0.4827586206896552
F1 score: 0.6153846153846154
[[ 28.   5.]
 [ 30. 195.]]


source

Precision: 0.7551020408163265
Recall: 0.34579439252336447
F1 score: 0.47435897435897434
[[ 37.  12.]
 [ 70. 139.]]


date

Precision: 0.9951923076923077
Recall: 0.9241071428571429
F1 score: 0.9583333333333334
[[207.   1.]
 [ 17.  33.]]


url

Precision: 1.0
Recall: 0.8
F1 score: 0.888888888888889
[[  4.   0.]
 [  1. 253.]]




## 3. Exparser


In [170]:
ground_truth =[]
#path = os.path.join("C:\\", "Users", "Tim", "Desktop", "bachelorarbeit", "goldstandards","ground_truth")
path = os.path.join("C:\\", "Users", "Tim", "Desktop", "bachelorarbeit", "goldstandards","ground_truth")
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r', encoding="utf8") as f: # open in readonly mode
        data = f.read()
        data = BeautifulSoup(data, "xml") 
        data = data.find_all("sequence")
        ground_truth.append(data)

exparser =[]
path = os.path.join("C:\\", "Users", "Tim", "Desktop", "bachelorarbeit", "goldstandards","exparser_xml")
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r', encoding="utf8") as f: # open in readonly mode
        data = f.read()
        data = BeautifulSoup(data, "xml") 
        data = data.find_all("sequence")
        exparser.append(data)
     

In [171]:
for i in range(13):
    if len(exparser[i]) != len(ground_truth[i]):
        print("error" + str(i))

In [172]:
#labels
for label in labels:
    lab_exparser =[]
    lab_ground =[]
    for val_exparser, val_ground in zip(exparser, ground_truth):
        for label_exparser, label_ground in zip(val_exparser,val_ground):
            tag_exp = label_exparser.find_all(label)
            tag_g = label_ground.find_all(label)
            
            
            for i in range(len(tag_exp)):
                if tag_exp[i].text == "":
                    tag_exp.remove(tag_exp[i]) 
            
            if len(tag_exp) != 0:
                if label == "editor" or label == "source":
                    lab_exparser.append(clean_edit("".join([word.text for word in tag_exp])))
                elif label == "location" or label == "publisher" or label == "author":
                    lab_exparser.append(clean_location("".join([word.text for word in tag_exp])))
                else:
                    lab_exparser.append(clean("".join([word.text for word in tag_exp])))
            else:
                lab_exparser.append("")
            
            if len(tag_g) != 0:
                if label == "editor" or label == "source":
                    lab_ground.append(clean_edit("".join([word.text for word in tag_g])))
                elif label == "location" or label == "publisher" or label == "author":
                    lab_ground.append(clean_location("".join([word.text for word in tag_g])))
                else:
                    lab_ground.append(clean("".join([word.text for word in tag_g])))
            else:
                lab_ground.append("")

    """
    for i,j in zip(lab_exparser, lab_ground):
        print("any " + i)
        print("gro " + j)
    """
    print(label+ "\n")
    confusion_matrix(lab_exparser,lab_ground)
    print("\n")

author

Precision: 0.8938547486033519
Recall: 0.7582938388625592
F1 score: 0.8205128205128205
[[160.  19.]
 [ 51.  28.]]


title

Precision: 0.9493670886075949
Recall: 0.6787330316742082
F1 score: 0.79155672823219
[[150.   8.]
 [ 71.  29.]]


publisher

Precision: 0.9661016949152542
Recall: 0.6477272727272727
F1 score: 0.7755102040816327
[[ 57.   2.]
 [ 31. 168.]]


location

Precision: 0.8451612903225807
Recall: 0.7751479289940828
F1 score: 0.808641975308642
[[131.  24.]
 [ 38.  65.]]


editor

Precision: 0.65
Recall: 0.6724137931034483
F1 score: 0.6610169491525424
[[ 39.  21.]
 [ 19. 179.]]


source

Precision: 0.8734177215189873
Recall: 0.6448598130841121
F1 score: 0.7419354838709676
[[ 69.  10.]
 [ 38. 141.]]


date

Precision: 0.9952380952380953
Recall: 0.9330357142857143
F1 score: 0.9631336405529954
[[209.   1.]
 [ 15.  33.]]


url

Precision: 1.0
Recall: 0.6
F1 score: 0.7499999999999999
[[  3.   0.]
 [  2. 253.]]


